In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

# from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('16')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [2]:
lines, groups = linesplit("""
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
""".strip())

In [3]:
lines, groups = linesplit("""
Valve XC has flow rate=0; tunnels lead to valves YK, AM
Valve ME has flow rate=0; tunnels lead to valves UU, SX
Valve EP has flow rate=0; tunnels lead to valves YS, QU
Valve GR has flow rate=0; tunnels lead to valves QZ, OG
Valve FA has flow rate=0; tunnels lead to valves DB, DP
Valve UJ has flow rate=0; tunnels lead to valves XN, CH
Valve QU has flow rate=0; tunnels lead to valves EP, YK
Valve OX has flow rate=19; tunnels lead to valves RI, PV
Valve VI has flow rate=0; tunnels lead to valves WI, XN
Valve IQ has flow rate=0; tunnels lead to valves QL, OG
Valve XO has flow rate=0; tunnels lead to valves GU, UI
Valve IY has flow rate=0; tunnels lead to valves VC, NT
Valve YS has flow rate=24; tunnel leads to valve EP
Valve XN has flow rate=7; tunnels lead to valves DG, UJ, VD, VI, OU
Valve AM has flow rate=6; tunnels lead to valves KA, NC, XC, TP, SI
Valve IH has flow rate=8; tunnels lead to valves TW, CH, WY, EC
Valve ZR has flow rate=18; tunnel leads to valve RI
Valve FP has flow rate=14; tunnels lead to valves DP, UF
Valve KA has flow rate=0; tunnels lead to valves VC, AM
Valve NC has flow rate=0; tunnels lead to valves UI, AM
Valve EC has flow rate=0; tunnels lead to valves IH, GU
Valve DG has flow rate=0; tunnels lead to valves AA, XN
Valve RI has flow rate=0; tunnels lead to valves OX, ZR
Valve NJ has flow rate=0; tunnels lead to valves YK, TW
Valve OG has flow rate=12; tunnels lead to valves GR, WY, IQ, UE
Valve IB has flow rate=0; tunnels lead to valves VB, UU
Valve RP has flow rate=0; tunnels lead to valves UI, OU
Valve OU has flow rate=0; tunnels lead to valves XN, RP
Valve NT has flow rate=0; tunnels lead to valves IY, AA
Valve MN has flow rate=0; tunnels lead to valves LX, VC
Valve SI has flow rate=0; tunnels lead to valves AM, AA
Valve VB has flow rate=0; tunnels lead to valves KT, IB
Valve UI has flow rate=4; tunnels lead to valves YI, XO, LX, NC, RP
Valve DL has flow rate=0; tunnels lead to valves GU, UE
Valve CH has flow rate=0; tunnels lead to valves UJ, IH
Valve WI has flow rate=0; tunnels lead to valves VI, VC
Valve GU has flow rate=11; tunnels lead to valves EC, XO, DL, SX
Valve KT has flow rate=17; tunnels lead to valves PV, VB
Valve TW has flow rate=0; tunnels lead to valves IH, NJ
Valve UE has flow rate=0; tunnels lead to valves DL, OG
Valve PV has flow rate=0; tunnels lead to valves KT, OX
Valve DP has flow rate=0; tunnels lead to valves FP, FA
Valve TP has flow rate=0; tunnels lead to valves VD, AM
Valve YI has flow rate=0; tunnels lead to valves AA, UI
Valve LX has flow rate=0; tunnels lead to valves UI, MN
Valve QZ has flow rate=0; tunnels lead to valves GR, UU
Valve DB has flow rate=23; tunnel leads to valve FA
Valve SX has flow rate=0; tunnels lead to valves ME, GU
Valve QL has flow rate=0; tunnels lead to valves AA, IQ
Valve YK has flow rate=16; tunnels lead to valves NJ, XC, QU
Valve VC has flow rate=5; tunnels lead to valves UF, KA, WI, IY, MN
Valve VD has flow rate=0; tunnels lead to valves TP, XN
Valve WY has flow rate=0; tunnels lead to valves IH, OG
Valve AA has flow rate=0; tunnels lead to valves YI, DG, QL, NT, SI
Valve UF has flow rate=0; tunnels lead to valves VC, FP
Valve UU has flow rate=15; tunnels lead to valves QZ, IB, ME
""".strip())

In [47]:
lines, groups = linesplit(raw_input, verbose=True)

64 lines
1 groups
>>>
Valve MU has flow rate=0; tunnels lead to valves VT, LA
Valve TQ has flow rate=0; tunnels lead to valves HU, SU
Valve YH has flow rate=0; tunnels lead to valves CN, BN
...


In [4]:
names = set()
rates = {}
neighs = {}

for line in lines:
  if m := re.findall(r'^Valve (.*) has flow rate=(.*); tunnels? leads? to valves? (.*)$', line):
    m = m[0]
    name = m[0]
    rate = int(m[1])
    neigh = m[2].split(', ')
    
    names.add(name)
    rates[name] = rate
    neighs[name] = neigh
  else:
    raise Exception()

In [207]:
paths = {}

for start in names:
  visited = set()
  distance = {start: 0}
  q = collections.deque([start])
  prev = {}
  
  while q:
    t = q.popleft()
    
    for tt in neighs[t]:
      if tt not in visited:
        visited.add(tt)
        q.append(tt)
        distance[tt] = distance[t] + 1
        prev[tt] = t
  
  for name in names:
    t = name
    path = []
    
    while t != start:
      path.append(t)
      t = prev[t]
    
    paths[(start, name)] = path[::-1]
    
byrate = sorted(names, key=lambda x: rates[x])
byrate

['QU',
 'YI',
 'VI',
 'VB',
 'RI',
 'GR',
 'PV',
 'KA',
 'VD',
 'TW',
 'DL',
 'MN',
 'XC',
 'SX',
 'FA',
 'UJ',
 'ME',
 'EC',
 'LX',
 'SI',
 'DP',
 'DG',
 'OU',
 'WY',
 'RP',
 'EP',
 'IB',
 'QZ',
 'IY',
 'QL',
 'IQ',
 'UF',
 'NT',
 'NC',
 'WI',
 'TP',
 'XO',
 'AA',
 'NJ',
 'CH',
 'UE',
 'UI',
 'VC',
 'AM',
 'XN',
 'IH',
 'GU',
 'OG',
 'FP',
 'UU',
 'YK',
 'KT',
 'ZR',
 'OX',
 'DB',
 'YS']

In [208]:
names

{'AA',
 'AM',
 'CH',
 'DB',
 'DG',
 'DL',
 'DP',
 'EC',
 'EP',
 'FA',
 'FP',
 'GR',
 'GU',
 'IB',
 'IH',
 'IQ',
 'IY',
 'KA',
 'KT',
 'LX',
 'ME',
 'MN',
 'NC',
 'NJ',
 'NT',
 'OG',
 'OU',
 'OX',
 'PV',
 'QL',
 'QU',
 'QZ',
 'RI',
 'RP',
 'SI',
 'SX',
 'TP',
 'TW',
 'UE',
 'UF',
 'UI',
 'UJ',
 'UU',
 'VB',
 'VC',
 'VD',
 'VI',
 'WI',
 'WY',
 'XC',
 'XN',
 'XO',
 'YI',
 'YK',
 'YS',
 'ZR'}

['II', 'FF', 'GG', 'AA', 'CC', 'EE', 'BB', 'DD', 'JJ', 'HH']

In [78]:
best = 0

for order in tqdm(itertools.permutations(byrate[-10:])):
  cur = 'AA'
  t = 30
  score = 0
  
  for i in order:
    t -= len(paths[(cur, i)])
    t -= 1
    if t <= 0:
      break
    score += rates[i] * t
    cur = i
  
  if score > best:
    # print(order)
    best = score
    print(best)

0it [00:00, ?it/s]

625
722
734
760
781
854
875
925
953
977
1010
1027
1043
1046
1055
1063
1075
1094
1121
1137
1140
1149
1154
1195
1199
1207
1219
1247
1312
1316
1328
1347
1361
1366
1377
1383
1431
1438
1487
1531
1571
1613
1653


In [66]:
submit1(1653)

Submitting 1653...


True

In [43]:
best = 0

for order in tqdm(itertools.permutations(byrate[-13:])):
  cur1 = 'AA'
  cur2 = 'AA'
  score = 0
  
  t = 26
  for i in order[::2]:
    t -= len(paths[(cur1, i)])
    t -= 1
    if t <= 0:
      break
    score += rates[i] * t
    cur1 = i
  
  t = 26
  for i in order[1::2]:
    t -= len(paths[(cur2, i)])
    t -= 1
    if t <= 0:
      break
    score += rates[i] * t
    cur2 = i
  
  if score > best:
    # print(order)
    best = score
    print(best)

0it [00:00, ?it/s]

797
805
845
859
866
898
899
906
955
975
976
987
1050
1063
1071
1073
1084
1089
1095
1102
1106
1110
1139
1212
1218
1228
1233
1242
1246
1247
1258
1320
1333
1341
1354
1355
1360
1363
1376
1381
1387
1388
1390
1396
1409
1414
1415
1417
1430
1435
1444
1446
1449
1455
1456
1460
1464
1474
1477
1481
1482
1485
1486
1496
1499
1503
1506
1513
1527
1549
1551
1552
1555
1557
1558
1567
1570
1588
1589
1601
1615
1625
1626
1628
1631
1636
1637
1640
1643
1646
1647
1655
1660
1662
1664
1670
1671


KeyboardInterrupt: 

In [ ]:
submit2(best)

In [11]:
import multiprocessing

In [21]:
best = 0

def proc_order(order):
  cur1 = 'AA'
  cur2 = 'AA'
  score = 0
  
  t = 26
  for i in order[::2]:
    t -= len(paths[(cur1, i)]) + 1
    if t <= 0:
      break
    score += rates[i] * t
    cur1 = i
  
  t = 26
  for i in order[1::2]:
    t -= len(paths[(cur2, i)]) + 1
    if t <= 0:
      break
    score += rates[i] * t
    cur2 = i
  
  return score

In [33]:
[(i, rates[i]) for i in byrate[-15:]]

[('RP', 3),
 ('RI', 4),
 ('VT', 5),
 ('JQ', 6),
 ('CN', 7),
 ('HR', 11),
 ('GG', 12),
 ('KV', 13),
 ('LI', 15),
 ('DT', 16),
 ('QF', 17),
 ('SU', 19),
 ('YA', 21),
 ('NS', 23),
 ('UH', 25)]

In [55]:
N = 15
M = 6

a = byrate[-N:]
best = 0

for i in tqdm(range(1 << N)):
  a1 = []
  a2 = []
  
  for j in range(N):
    if i & (1 << j) > 0:
      a1.append(a[j])
    else:
      a2.append(a[j])
      
  if len(a1) <= M or len(a2) <= M:
    continue
  
  best1 = 0
  best2 = 0
  
  for order in itertools.permutations(a1):
    cur1 = 'AA'
    score = 0

    t = 26
    for i in order:
      t -= len(paths[(cur1, i)]) + 1
      if t <= 0:
        break
      score += rates[i] * t
      cur1 = i
    
    if score > best1:
      best1 = score
  
  for order in itertools.permutations(a2):
    cur1 = 'AA'
    score = 0

    t = 26
    for i in order:
      t -= len(paths[(cur1, i)]) + 1
      if t <= 0:
        break
      score += rates[i] * t
      cur1 = i
    
    if score > best2:
      best2 = score
  
  if best1 + best2 > best:
    best = best1 + best2
    print(best)

  0%|          | 0/32768 [00:00<?, ?it/s]

1715
1763
1892
1978
1981
2032
2074
2083
2091
2098
2121
2136
2145
2200
2218
2223


In [52]:
submit2(best)

Submitting 2210...
[<p>That's not the right answer.  If you're stuck, make sure you're using the full input data; there are also some general tips on the <a href="/2022/about">about page</a>, or you can ask for hints on the <a href="https://www.reddit.com/r/adventofcode/" target="_blank">subreddit</a>.  Because you have guessed incorrectly 5 times on this puzzle, please wait 5 minutes before trying again. (You guessed <span style="white-space:nowrap;"><code>2210</code>.)</span> <a href="/2022/day/16">[Return to Day 16]</a></p>]


/Users/abra/projects/jupyter/adventofcode2022/adventlib/common.py:51: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 51 of the file /Users/abra/projects/jupyter/adventofcode2022/adventlib/common.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  print(BeautifulSoup(response.text).select('main>article>p'))


AssertionError: 

In [57]:
N = 15
M = 6

a = byrate[-N:]
best = 0

for i in tqdm(range(1 << N)):
  a1 = []
  a2 = []
  
  for j in range(N):
    if i & (1 << j) > 0:
      a1.append(a[j])
    else:
      a2.append(a[j])
      
  if len(a1) <= M or len(a2) <= M or len(a1) < len(a2):
    continue
  
  best1 = 0
  best2 = 0
  
  for order in itertools.permutations(a1):
    cur1 = 'AA'
    score = 0

    t = 26
    for i in order:
      t -= len(paths[(cur1, i)]) + 1
      if t <= 0:
        break
      score += rates[i] * t
      cur1 = i
    
    if score > best1:
      best1 = score
  
  for order in itertools.permutations(a2):
    cur1 = 'AA'
    score = 0

    t = 26
    for i in order:
      t -= len(paths[(cur1, i)]) + 1
      if t <= 0:
        break
      score += rates[i] * t
      cur1 = i
    
    if score > best2:
      best2 = score
  
  if best1 + best2 > best:
    best = best1 + best2
    print(best, a1, a2)

  0%|          | 0/32768 [00:00<?, ?it/s]

1672 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'KV'] ['LI', 'DT', 'QF', 'SU', 'YA', 'NS', 'UH']
1757 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'LI'] ['KV', 'DT', 'QF', 'SU', 'YA', 'NS', 'UH']
1828 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'DT'] ['KV', 'LI', 'QF', 'SU', 'YA', 'NS', 'UH']
1981 ['RP', 'RI', 'VT', 'JQ', 'CN', 'GG', 'LI', 'DT'] ['HR', 'KV', 'QF', 'SU', 'YA', 'NS', 'UH']
2004 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'SU'] ['KV', 'LI', 'DT', 'QF', 'YA', 'NS', 'UH']
2032 ['RP', 'RI', 'VT', 'JQ', 'CN', 'GG', 'LI', 'SU'] ['HR', 'KV', 'DT', 'QF', 'YA', 'NS', 'UH']
2098 ['RP', 'RI', 'VT', 'JQ', 'CN', 'GG', 'DT', 'SU'] ['HR', 'KV', 'LI', 'QF', 'YA', 'NS', 'UH']
2121 ['RP', 'RI', 'VT', 'JQ', 'GG', 'LI', 'DT', 'SU'] ['CN', 'HR', 'KV', 'QF', 'YA', 'NS', 'UH']
2136 ['RP', 'RI', 'JQ', 'CN', 'GG', 'LI', 'DT', 'SU'] ['VT', 'HR', 'KV', 'QF', 'YA', 'NS', 'UH']
2145 ['RP', 'JQ', 'CN', 'GG', 'LI', 'DT', 'QF', 'SU'] ['RI', 'VT', 'HR', 'KV', 'YA', 'NS', 'UH']
2182 ['RP', 'RI', 'VT', 'JQ', 

KeyboardInterrupt: 

In [60]:
best1 = 0
best2 = 0
best1_order = None
best2_order = None

a1 = ['RP', 'VT', 'CN', 'HR', 'KV', 'DT', 'YA', 'NS']
a2 = ['RI', 'JQ', 'GG', 'LI', 'QF', 'SU', 'UH']

for order in itertools.permutations(a1):
  cur1 = 'AA'
  score = 0

  t = 26
  for i in order:
    t -= len(paths[(cur1, i)]) + 1
    if t <= 0:
      break
    score += rates[i] * t
    cur1 = i

  if score > best1:
    best1 = score
    best1_order = order
    print('A', order, best1)

for order in itertools.permutations(a2):
  cur1 = 'AA'
  score = 0

  t = 26
  for i in order:
    t -= len(paths[(cur1, i)]) + 1
    if t <= 0:
      break
    score += rates[i] * t
    cur1 = i

  if score > best2:
    best2 = score
    best2_order = order
    print('A', order, best2)

A ('RP', 'VT', 'CN', 'HR', 'KV', 'DT', 'YA', 'NS') 310
A ('RP', 'VT', 'CN', 'HR', 'NS', 'KV', 'DT', 'YA') 333
A ('RP', 'VT', 'CN', 'KV', 'HR', 'DT', 'YA', 'NS') 390
A ('RP', 'VT', 'CN', 'KV', 'DT', 'HR', 'YA', 'NS') 405
A ('RP', 'VT', 'CN', 'KV', 'NS', 'HR', 'DT', 'YA') 460
A ('RP', 'VT', 'CN', 'KV', 'NS', 'YA', 'HR', 'DT') 470
A ('RP', 'VT', 'HR', 'KV', 'NS', 'YA', 'CN', 'DT') 486
A ('RP', 'VT', 'HR', 'YA', 'NS', 'CN', 'KV', 'DT') 521
A ('RP', 'VT', 'HR', 'YA', 'NS', 'KV', 'CN', 'DT') 534
A ('RP', 'VT', 'HR', 'NS', 'KV', 'DT', 'CN', 'YA') 547
A ('RP', 'VT', 'HR', 'NS', 'YA', 'CN', 'KV', 'DT') 571
A ('RP', 'VT', 'KV', 'HR', 'YA', 'CN', 'DT', 'NS') 579
A ('RP', 'VT', 'KV', 'HR', 'YA', 'NS', 'CN', 'DT') 648
A ('RP', 'VT', 'KV', 'HR', 'NS', 'YA', 'CN', 'DT') 698
A ('RP', 'VT', 'KV', 'YA', 'NS', 'HR', 'CN', 'DT') 703
A ('RP', 'VT', 'KV', 'NS', 'HR', 'YA', 'CN', 'DT') 768
A ('RP', 'VT', 'KV', 'NS', 'YA', 'HR', 'CN', 'DT') 808
A ('RP', 'KV', 'NS', 'HR', 'YA', 'VT', 'CN', 'DT') 809
A ('RP', '

In [63]:
cur1 = 'AA'
score = 0

t = 26
for i in best1_order:
  t -= len(paths[(cur1, i)]) + 1
  if t <= 0:
    break
  score += rates[i] * t
  cur1 = i
  print('used', cur1, score, t)

used KV 273 21
used NS 687 18
used YA 1002 15
used HR 1123 11
used DT 1187 4


In [64]:
cur1 = 'AA'
score = 0

t = 26
for i in best2_order:
  t -= len(paths[(cur1, i)]) + 1
  if t <= 0:
    break
  score += rates[i] * t
  cur1 = i
  print('used', cur1, score, t)

used RI 92 23
used GG 332 20
used SU 655 17
used JQ 733 13
used QF 886 9
used UH 1036 6


In [66]:
best1_order

('KV', 'NS', 'YA', 'HR', 'DT', 'RP', 'VT', 'CN')

In [71]:
calc_with_order(set(best1_order), 'AA', 26, 0)

1187

In [70]:
def calc_with_order(options, cur, timeleft, score):
  best = score

  for option in list(options):
    time = timeleft - len(paths[(cur, option)]) - 1
    if time <= 0:
      continue
    
    gain_score = rates[option] * time
    
    options.remove(option)
    best = max(best, calc_with_order(options, option, time, score + gain_score))
    options.add(option)
  
  return best

In [151]:
N = 15
M = 6

a = byrate[-N:]
best = 0

for i in tqdm(range(1 << N)):
  a1 = []
  a2 = []
  
  for j in range(N):
    if i & (1 << j) > 0:
      a1.append(a[j])
    else:
      a2.append(a[j])
      
  if len(a1) <= M or len(a2) <= M or len(a1) < len(a2):
    continue
  
  best1 = calc_with_order(set(a1), 'AA', 26, 0)
  best2 = calc_with_order(set(a2), 'AA', 26, 0)
  
  if best1 + best2 > best:
    best = best1 + best2
    print(best, a1, a2)

  0%|          | 0/32768 [00:00<?, ?it/s]

1672 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'KV'] ['LI', 'DT', 'QF', 'SU', 'YA', 'NS', 'UH']
1757 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'LI'] ['KV', 'DT', 'QF', 'SU', 'YA', 'NS', 'UH']
1828 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'DT'] ['KV', 'LI', 'QF', 'SU', 'YA', 'NS', 'UH']
1981 ['RP', 'RI', 'VT', 'JQ', 'CN', 'GG', 'LI', 'DT'] ['HR', 'KV', 'QF', 'SU', 'YA', 'NS', 'UH']
2004 ['RP', 'RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'SU'] ['KV', 'LI', 'DT', 'QF', 'YA', 'NS', 'UH']
2032 ['RP', 'RI', 'VT', 'JQ', 'CN', 'GG', 'LI', 'SU'] ['HR', 'KV', 'DT', 'QF', 'YA', 'NS', 'UH']
2098 ['RP', 'RI', 'VT', 'JQ', 'CN', 'GG', 'DT', 'SU'] ['HR', 'KV', 'LI', 'QF', 'YA', 'NS', 'UH']
2121 ['RP', 'RI', 'VT', 'JQ', 'GG', 'LI', 'DT', 'SU'] ['CN', 'HR', 'KV', 'QF', 'YA', 'NS', 'UH']
2136 ['RP', 'RI', 'JQ', 'CN', 'GG', 'LI', 'DT', 'SU'] ['VT', 'HR', 'KV', 'QF', 'YA', 'NS', 'UH']
2145 ['RP', 'JQ', 'CN', 'GG', 'LI', 'DT', 'QF', 'SU'] ['RI', 'VT', 'HR', 'KV', 'YA', 'NS', 'UH']
2182 ['RP', 'RI', 'VT', 'JQ', 

In [80]:
sum(array(list(rates.values())) >= 0), sum(array(list(rates.values())) > 0)

(64, 15)

In [127]:
name_list = list(names)
name_to_index = {j: i for i, j in enumerate(names)}

neighs_i = [
  [name_to_index[j] for j in neighs[i]]
  for i in name_list
]
rates_i = [rates[i] for i in name_list]

useful_nodes = [i for i, j in enumerate(rates_i) if j > 0]

N = len(name_list)
M = len(useful_nodes)
masks = arange(1 << M, dtype=int)

dp = zeros((N, N, 1 << M), dtype=int)
dp -= 1000000
dp[0][0][0] = 0

for time_remaining in tqdm(range(30, 0, -1)):
# for time_remaining in tqdm(range(26, 20, -1)):
  newdp = dp + 0
  
  # movement
  for v1 in range(N):
    for v2 in neighs_i[v1]:
      newdp[v2, :, :] = np.maximum(dp[v1, :, :], newdp[v2, :, :])
      newdp[:, v2, :] = np.maximum(dp[:, v1, :], newdp[:, v2, :])
    
  # activation
  for i1, v1 in enumerate(useful_nodes):
    m1 = where((masks & (1 << i1)) == 0)[0]
    m2 = m1 + (1 << i1)
    
    newdp[v1, :, m2] = np.maximum(
      dp[v1, :, m1] + (time_remaining - 1) * rates_i[v1],
      newdp[v1, :, m2])
    newdp[:, v1, m2] = np.maximum(
      dp[:, v1, m1] + (time_remaining - 1) * rates_i[v1],
      newdp[:, v1, m2])

  dp = newdp
  print(np.max(dp))

  0%|          | 0/30 [00:00<?, ?it/s]

174
174


KeyboardInterrupt: 

In [125]:
x = np.max(dp, axis=(0, 1))
best = 0
for i in tqdm(range(1 << M)):
  m = where((masks & i) == 0)
  best = max(best, np.max(x[m] + x[i]))
best

  0%|          | 0/32768 [00:00<?, ?it/s]

2616

In [6]:
name_list = list(names)
name_to_index = {j: i for i, j in enumerate(names)}

neighs_i = [
  [name_to_index[j] for j in neighs[i]]
  for i in name_list
]
rates_i = [rates[i] for i in name_list]

useful_nodes = [i for i, j in enumerate(rates_i) if j > 0]

N = len(name_list)
M = len(useful_nodes)
masks = arange(1 << M, dtype=int)

dp = full((N, 1 << M), -100000, dtype=int)
dp[name_to_index['AA'], 0] = 0

for time_remaining in range(30, 0, -1):
  newdp = np.array(dp)
  
  # movement
  for v1 in range(N):
    for v2 in neighs_i[v1]:
      newdp[v2, :] = np.maximum(newdp[v2, :], dp[v1, :])
    
  # activation
  for i1, v1 in enumerate(useful_nodes):
    m1 = where((masks & (1 << i1)) == 0)[0]
    m2 = m1 | (1 << i1)
    
    newdp[v1, m2] = np.maximum(
      newdp[v1, m2],
      dp[v1, m1] + (time_remaining - 1) * rates_i[v1])

  dp = newdp
  
print('ans 1', np.max(dp))

ans 1 1460


In [211]:
dp = full((N, 1 << M), -100000, dtype=int)
dp[name_to_index['AA'], 0] = 0

for time_remaining in range(26, 0, -1):
  newdp = np.array(dp)
  
  # movement
  for v1 in range(N):
    for v2 in neighs_i[v1]:
      newdp[v2, :] = np.maximum(newdp[v2, :], dp[v1, :])
    
  # activation
  for i1, v1 in enumerate(useful_nodes):
    m1 = where((masks & (1 << i1)) == 0)[0]
    m2 = m1 | (1 << i1)
    
    newdp[v1, m2] = np.maximum(
      newdp[v1, m2],
      dp[v1, m1] + (time_remaining - 1) * rates_i[v1])

  dp = newdp

x = np.max(dp, axis=0)
best = 0
for i in range(1 << M):
  best = max(best, np.max(x[i] + x[(masks & i) == 0]))

print('ans 2', best)

ans 2 2117


In [5]:
# names = set of 'AA' names
# neighs = dict of lists of neighboring names
# rates = dict of integer pressures

# convert to indices for convenience
name_list = list(names)
name_to_index = {j: i for i, j in enumerate(names)}
neighs_i = [
  [name_to_index[j] for j in neighs[i]]
  for i in name_list
]
rates_i = [rates[i] for i in name_list]

useful_nodes = [i for i, j in enumerate(rates_i) if j > 0]

N = len(name_list)
M = len(useful_nodes)
masks = np.arange(1 << M, dtype=int)
  
dp = np.full((N, 1 << M), -100000, dtype=int)
dp[name_to_index['AA'], 0] = 0

for time_remaining in range(26, 0, -1):
  newdp = np.array(dp)
  
  # movement
  for v1 in range(N):
    for v2 in neighs_i[v1]:
      newdp[v2, :] = np.maximum(newdp[v2, :], dp[v1, :])
    
  # activation
  for i1, v1 in enumerate(useful_nodes):
    mask_unused = where((masks & (1 << i1)) == 0)[0]
    mask_used = mask_unused | (1 << i1)
    
    newdp[v1, mask_used] = np.maximum(
      newdp[v1, mask_used],
      dp[v1, mask_unused] + (time_remaining - 1) * rates_i[v1])

  dp = newdp

best_for_mask = np.max(dp, axis=0)
best = 0

# extend masks
for i in range(1 << M):
  for j in range(M):
    best_for_mask[i | (1 << j)] = max(best_for_mask[i | (1 << j)], best_for_mask[i])

print(np.max(best_for_mask[masks] + best_for_mask[(1 << M) - masks - 1]))

2117
